# Use the counter prototype with OTAnalytics

> Note: The base path is set in `.vscode/settings.json` to workspace path. Otherwise Jupyter can't import OTAnalytics.

## Import the prototype modules

Before working with events, you need to import the event_processor to import and convert the events to a shaped data frame.

In [ ]:
# Import libraries and modules
# OTAnalytics modules
from OTAnalytics.plugin_prototypes.event_parser.event_parser import EventParser
from OTAnalytics.plugin_prototypes.counter.counter import Counter

%load_ext autoreload
%autoreload 2

## Set config variables

In the current version, configs for prototypes are stored in dictionaries within the Jupyter notebook. In a future version, the will likely be stored in a config file.

The path to the eventlists can be a path to a single event list or a folder (all *.otevent files within this folder will be imported).

At each section, the counts will be calculated by direction using the flows. Therefore, you have to name the directions (default: "in" and "out"). "first_to_last_section" names the direction in which the given section is intersected first by the flow. "last_to_first_section" names the direction in which the given section is intersected last.

In [ ]:
# Set env parameters and path
CONFIG = {
    "TIME_FORMAT": "%d.%m%.%y %H:%M Uhr",
    "FILTER_CLASS": [],
    "FILTER_SECTION": [],
    "EVENTLIST_PATH": "<YOUR PATH>",
    "SECTIONSLIST_PATH": "<YOUR PATH>",
    "FROM_TIME": "2023-03-28 00:00:00",
    "TO_TIME": "2023-03-29 00:00:00",
    "INTERVAL_LENGTH_MIN": 15,  # in minutes
    "DIRECTION_NAMES": {
        "first_to_last_section": "in",
        "last_to_first_section": "out",
    },
}

## Import Events
The event parser is used to convert the events from the zipped json format of an *.otevent file to a pandas dataframe.

In [ ]:
event_processor = EventParser(CONFIG)
events = event_processor.process_events()

## Create Count Table and Plot Counts

The classes of road users, sections and directions that should be included in the counting table and counting plot can be filtered. If no filter is provided, all elements will be shown.

In [ ]:
counts_processor = Counter(CONFIG, events)

filter_sections = []
filter_directions = []
filter_classes = [] 

counts_table = counts_processor.create_counting_table(
    filter_sections, filter_directions, filter_classes
)
counts_processor.plot_counts()

## Define flows
In the counting table, only overall counts at a cross section are returned. If you seek to analyze the flows of e.g. an intersection, you have to calculate flows. A flow is defined as a movement between the first and the last intersected section.

The flows are defined in an dict with a structure as can be seen in the `flow_names` dict. Furthermore, for each flow, the classes that should be analyzed can be specified. Using a mapping dict (like the `mode_mapper` dict), the original names of the classes can be replaced by cutsom names for customized outputs.

In [ ]:
classes_street = [
    "bicyclist",
    "bicyclist_with_trailer",
    "cargobike_driver",
    "car",
    "car_with_trailer",
    "bus",
    "motorcyclist",
    "delivery_van_with_trailer",
    "delivery_van",
    "private_van",
    "private_van_with_trailer",
    "truck",
    "truck_with_trailer",
    "truck_with_semitrailer",
]

classes_ped = ["pedestrian", "bicyclist", "bicyclist_with_trailer", "cargobike_driver"]

flow_names = {
    "Strom 01": {
        "classes": classes_street,
        "from": ["B75_West"],
        "to": ["Kroegersweg"],
    },
    "Strom 02": {"classes": classes_street, "from": ["B75_West"], "to": ["B75_Ost"]},
    "Strom 03": {"classes": classes_street, "from": ["B75_West"], "to": ["Rothenmoor"]},
    "Strom 04": {"classes": classes_street, "from": ["Rothenmoor"], "to": ["B75_West"]},
    "Strom 05": {
        "classes": classes_street,
        "from": ["Rothenmoor"],
        "to": ["Kroegersweg"],
    },
    "Strom 06": {"classes": classes_street, "from": ["Rothenmoor"], "to": ["B75_Ost"]},
    "Strom 07": {"classes": classes_street, "from": ["B75_Ost"], "to": ["Rothenmoor"]},
    "Strom 08": {"classes": classes_street, "from": ["B75_Ost"], "to": ["B75_West"]},
    "Strom 09": {"classes": classes_street, "from": ["B75_Ost"], "to": ["Kroegersweg"]},
    "Strom 10": {"classes": classes_street, "from": ["Kroegersweg"], "to": ["B75_Ost"]},
    "Strom 11": {
        "classes": classes_street,
        "from": ["Rothenmoor"],
        "to": ["Rothenmoor"],
    },
    "Strom 12": {
        "classes": classes_street,
        "from": ["Kroegersweg"],
        "to": ["B75_West"],
    },
    "Strom Q 1/3 S": {
        "classes": classes_ped,
        "from": ["Kroegersweg"],
        "to": ["Rothenmoor"],
    },
    "Strom Q 1/3 N": {
        "classes": classes_ped,
        "from": ["Rothenmoor"],
        "to": ["Kroegersweg"],
    },
    "Strom Q 2 O": {
        "classes": classes_ped,
        "from": ["B75_West_PED_BIKE"],
        "to": ["B75_Ost_PED_BIKE"],
    },
    "Strom Q 2 W": {
        "classes": classes_ped,
        "from": ["B75_Ost_PED_BIKE"],
        "to": ["B75_West_PED_BIKE"],
    },
}

mode_mapper = {
    "pedestrian": "Fußgänger",
    "bicyclist": "Radfahrer",
    "bicyclist_with_trailer": "Radfahrer",
    "cargobike_driver": "Radfahrer",
    "car": "Pkw",
    "car_with_trailer": "Pkw mit Anhänger",
    "bus": "Bus",
    "motorcyclist": "Motorrad",
    "delivery_van": "Lieferwagen bis 3,5t",
    "delivery_van_with_trailer": "Lieferwagen bis 3,5t",
    "private_van": "Pkw",
    "private_van_with_trailer": "Pkw mit Anhänger",
    "truck": "Lkw",
    "truck_with_trailer": "Lkw mit Anhänger",
    "truck_with_semitrailer": "Sattelkraftfahrzeuge (Lastzüge)",
}

## Create Flow Table

In [ ]:
flow_processor = Counter(CONFIG, events)

filter_sections = []
filter_classes = []

flows = flow_processor.get_flows(filter_sections, filter_classes)
flow_table = flow_processor.create_flow_table(filter_sections, filter_classes)

## Convert Flow Table
The classes of road users, sections and directions that should be included in the flow table and flow plot can be filtered. If no filter is provided, all elements will be shown.

In [ ]:
flows_export = flow_processor.convert_flow_table(flows, flow_names, mode_mapper)
flow_table_export = flow_processor.convert_flow_table(
    flow_table, flow_names, mode_mapper, aggregated=True
)

## Plot Flows

In [ ]:
flow_processor.plot_flows(flow_table_export)

## Export Flow tables

In [ ]:
flows_export.to_csv(CONFIG["EVENTLIST_PATH"] + "/results.csv", index=False)
flow_table_export.to_csv(
    CONFIG["EVENTLIST_PATH"] + "/results_agg_15min.csv", index=False
)